In [5]:
# КРОК 0. Імпорт необхідних бібліотек
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# КРОК 1. Завантаження даних
import pandas as pd

# Specify the path to your CSV file
csv_file_path = "housing.csv"

# Load the CSV file
data = pd.read_csv(csv_file_path)

# Display the first few rows
print(data.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB
None


In [10]:
# Specify the target column (e.g., "target_column")
target_column = "median_house_value"

# Split into features (X) and target (y)
X = data.drop(columns=[target_column,'ocean_proximity'])  # Features
y = data[target_column]                # Target

# Display the shapes of X and y
print(f"Features (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")


Features (X) shape: (20640, 8)
Target (y) shape: (20640,)


In [11]:

# Перевіримо розміри
print("Форма X:", X.shape)
print("Форма y:", y.shape)


Форма X: (20640, 8)
Форма y: (20640,)


In [13]:

# КРОК 2. Попередня обробка даних
# 2.1. Розділяємо дані на тренувальний і тестовий набори
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 2.2. Нормалізація ознак
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# КРОК 3. Побудова моделі
# Послідовна модель (MLP) з кількома Dense-шарами
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Вихід: 1 нейрон для регресії

# КРОК 4. Компіляція
model.compile(
    optimizer='adam', 
    loss='mse',        # Середньоквадратична помилка
    metrics=['mae']    # Середня абсолютна похибка
)


c:\Users\Eve8\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:

# КРОК 5. Навчання (фітинг)
history = model.fit(
    X_train_scaled, y_train, 
    validation_split=0.2,    # виділяємо 20% від тренувальних даних на валідацію
    epochs=20,               # кількість епох
    batch_size=32,           # розмір міні-вибірки
    verbose=1                # виводимо прогрес навчання
)


Epoch 1/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 54924431360.0000 - mae: 204986.7656 - val_loss: 56500260864.0000 - val_mae: 207094.2969
Epoch 2/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 54877822976.0000 - mae: 204328.5469 - val_loss: 53775372288.0000 - val_mae: 201073.8594
Epoch 3/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 51520565248.0000 - mae: 196555.7656 - val_loss: 47576412160.0000 - val_mae: 186614.2344
Epoch 4/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 44680011776.0000 - mae: 180286.9062 - val_loss: 38845972480.0000 - val_mae: 164515.5625
Epoch 5/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 35761242112.0000 - mae: 156644.0781 - val_loss: 29622013952.0000 - val_mae: 138543.5469
Epoch 6/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 27095889920.0000 - mae: 130951.3047 - val_loss: 21776781312.0000 - val_mae: 113133.6719
Epoch 7/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 19788902400.0000 - mae: 106868.4141 - val_loss: 1639

In [15]:

# КРОК 6. Оцінка на тестових даних
test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test MSE: {test_loss:.4f}")
print(f"Test MAE: {test_mae:.4f}")


Test MSE: 9272782848.0000
Test MAE: 67436.8984


In [18]:
y_test.shape

(4128,)

In [19]:

# КРОК 7. Приклад прогнозу
predictions = model.predict(X_test_scaled[:5])
print("Справжні значення:", y_test[:5])
print("Прогнози моделі:   ", predictions.reshape(-1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Справжні значення: 20046     47700.0
3024      45800.0
15663    500001.0
20484    218600.0
9814     278000.0
Name: median_house_value, dtype: float64
Прогнози моделі:    [451.66388 451.66388 451.66388 451.66388 451.66388]
